# ReAct agent

**Autores:** [Lucas Lima](https://github.com/lucasouzamil) e [Luiz Eduardo Pini](https://github.com/luizehp)

**Vídeo de explicação:** [YouTube](https://youtu.be/awaqm7JQ99g)

**Baseado no artigo** [Multi-agent System Design Patterns From Scratch In Python | ReAct Agents](https://medium.com/aimonks/multi-agent-system-design-patterns-from-scratch-in-python-react-agents-e4480d099f38)

**Nossas modificações:** Alteramos para usar o modelo do Gemini e também alteramos o prompt original para aceitar o scratchpad no final. Dessa forma o agente pode funcionar em sessões diferentes, ao invés de permanecer em uma única.

In [164]:
from typing import Callable, Dict, Any
import json

def extract_function_metadata(function: Callable) -> Dict[str, Any]:
    """
    Creates a metadata dictionary from a function's signature.
    
    Parameters:
        function: The target function to analyze
        
    Returns:
        A dictionary containing the function's name, documentation, and parameter specifications
    """
    # Initialize the basic structure
    metadata = {
        "name": function.__name__,
        "description": function.__doc__,
        "parameters": {"properties": {}}
    }
    
    # Extract parameter types (excluding return annotation)
    parameter_types = {
        param_name: {"type": param_type.__name__}
        for param_name, param_type in function.__annotations__.items()
        if param_name != "return"
    }
    
    # Add parameters to metadata
    metadata["parameters"]["properties"] = parameter_types
    return metadata

def convert_argument_types(tool_invocation: Dict[str, Any], function_spec: Dict[str, Any]) -> Dict[str, Any]:
    """
    Ensures all arguments match their expected types according to the function specification.
    
    Parameters:
        tool_invocation: Dictionary with the tool name and arguments
        function_spec: Dictionary containing the expected parameter types
        
    Returns:
        Updated tool invocation with correctly typed arguments
    """
    expected_params = function_spec["parameters"]["properties"]
    
    # Type conversion mapping
    type_converters = {
        "int": int,
        "str": str,
        "bool": bool,
        "float": float
    }
    
    # Convert each argument to its expected type if needed
    for arg_name, arg_value in tool_invocation["arguments"].items():
        target_type = expected_params[arg_name].get("type")
        if not isinstance(arg_value, type_converters[target_type]):
            tool_invocation["arguments"][arg_name] = type_converters[target_type](arg_value)
            
    return tool_invocation

class Tool:
    """
    Wrapper class that encapsulates a function as a callable tool.
    
    Attributes:
        name: Tool identifier
        function: The underlying function
        specification: JSON-formatted function metadata
    """
    def __init__(self, name: str, function: Callable, specification: str):
        self.name = name
        self.function = function
        self.specification = specification
        
    def __str__(self):
        return self.specification
        
    def execute(self, **kwargs):
        """
        Runs the wrapped function with the provided arguments.
        
        Parameters:
            **kwargs: Arguments to pass to the function
            
        Returns:
            The function's result
        """
        return self.function(**kwargs)

def tool(function: Callable) -> Tool:
    """
    Decorator that transforms a regular function into a Tool instance.
    
    Parameters:
        function: The function to convert into a tool
        
    Returns:
        A fully configured Tool object
    """
    def create_tool():
        metadata = extract_function_metadata(function)
        return Tool(
            name=metadata.get("name"),
            function=function,
            specification=json.dumps(metadata)
        )
    
    return create_tool()

In [165]:
import json
import re
from typing import List, Dict, Any, Callable
from colorama import init, Fore, Style

# Initialize colorama
init(autoreset=True)

class ToolCallingAgent:
    """
    An agent that integrates language models with function-calling tools.
    
    This class manages the interaction between a language model and a set of tools,
    allowing the model to decide when to call functions and processing the results.
    """
    
    def __init__(self, llm: Callable, system_prompt: str, tools: List[Any]):
        """
        Initialize an AI agent with a language model and tools.
        
        Args:
            llm: Function that takes a prompt string and returns a response
            system_prompt: Instructions for guiding the language model's behavior
            tools: List of Tool objects that the agent can use
        """
        self.model = llm
        self.system_prompt = system_prompt
        self.tools = {tool.name: tool for tool in tools}
        self.conversation_history = []
        print(f"{Fore.GREEN}ToolCallingAgent initialized with {len(tools)} tools{Style.RESET_ALL}")
        
    def format_tools_for_prompt(self) -> str:
        """Format all available tools into a format the language model can understand."""
        tools_json = []
        for tool in self.tools.values():
            try:
                # Use the specification provided by the @tool decorator
                tools_json.append(json.loads(tool.specification))
            except (json.JSONDecodeError, AttributeError):
                # Fallback for tools without proper specification
                tools_json.append({
                    "name": tool.name,
                    "description": getattr(tool, "description", "No description available"),
                    "parameters": {"properties": {}}
                })
        
        print(f"{Fore.CYAN}Formatted {len(tools_json)} tools for LLM prompt{Style.RESET_ALL}")
        return f"<tools>\n{json.dumps(tools_json, indent=2)}\n</tools>"
    
    def extract_tool_calls(self, response: str) -> List[Dict[str, Any]]:
        """
        Parse the language model's response to extract tool call requests.
        
        Args:
            response: The text response from the language model
            
        Returns:
            A list of tool call dictionaries with 'name' and 'arguments' keys
        """
        tool_calls = []
        pattern = r"<tool_call>(.*?)</tool_call>"
        matches = re.findall(pattern, response, re.DOTALL)
        
        for match in matches:
            try:
                tool_call = json.loads(match.strip())
                if "name" in tool_call and "arguments" in tool_call:
                    tool_calls.append(tool_call)
            except json.JSONDecodeError:
                continue
        
        if tool_calls:
            print(f"{Fore.YELLOW}Extracted {len(tool_calls)} tool call(s) from LLM response{Style.RESET_ALL}")
        else:
            print(f"{Fore.YELLOW}No tool calls found in LLM response{Style.RESET_ALL}")
                
        return tool_calls
    
    def execute_tool(self, tool_call: Dict[str, Any]) -> Any:
        """
        Execute a tool based on the model's request.
        
        Args:
            tool_call: Dictionary with 'name' and 'arguments' for the tool
            
        Returns:
            The result from executing the tool
        """
        tool_name = tool_call.get("name")
        arguments = tool_call.get("arguments", {})
        
        if tool_name not in self.tools:
            print(f"{Fore.RED}Error: Tool '{tool_name}' not found{Style.RESET_ALL}")
            return f"Error: Tool '{tool_name}' not found"
            
        tool = self.tools[tool_name]
        print(f"{Fore.MAGENTA}Executing tool: {tool_name} with arguments: {json.dumps(arguments)}{Style.RESET_ALL}")
        
        # Validate and convert argument types using the tool's specification
        try:
            if hasattr(tool, "specification"):
                tool_spec = json.loads(tool.specification)
                validated_args = self.convert_argument_types(
                    {"arguments": arguments}, 
                    tool_spec
                )["arguments"]
                arguments = validated_args
                print(f"{Fore.BLUE}Arguments validated and converted to appropriate types{Style.RESET_ALL}")
        except (json.JSONDecodeError, AttributeError, KeyError) as e:
            print(f"{Fore.RED}Error validating arguments: {str(e)}{Style.RESET_ALL}")
            # Continue with original arguments if validation fails
            pass
            
        try:
            # Handle execution based on the tool interface
            # First try the execute method for tools created with the @tool decorator
            if hasattr(tool, "execute"):
                print(f"{Fore.GREEN}Calling tool.execute() method{Style.RESET_ALL}")
                return tool.execute(**arguments)
            # Then try the function attribute which is used by the @tool decorator
            elif hasattr(tool, "function"):
                print(f"{Fore.GREEN}Calling tool.function() method{Style.RESET_ALL}")
                return tool.function(**arguments)
            # Fall back to run method
            elif hasattr(tool, "run"):
                print(f"{Fore.GREEN}Calling tool.run() method{Style.RESET_ALL}")
                return tool.run(**arguments)
            # Last resort: call the tool directly if it's callable
            elif callable(tool):
                print(f"{Fore.GREEN}Calling tool directly{Style.RESET_ALL}")
                return tool(**arguments)
            else:
                print(f"{Fore.RED}Error: Tool '{tool_name}' is not callable{Style.RESET_ALL}")
                return f"Error: Tool '{tool_name}' is not callable"
        except Exception as e:
            print(f"{Fore.RED}Error executing {tool_name}: {str(e)}{Style.RESET_ALL}")
            return f"Error executing {tool_name}: {str(e)}"
    
    def convert_argument_types(self, tool_call: Dict[str, Any], tool_spec: Dict[str, Any]) -> Dict[str, Any]:
        """
        Convert arguments to their expected types based on tool specification.
        
        Args:
            tool_call: Dictionary containing arguments to convert
            tool_spec: Tool specification with expected types
            
        Returns:
            Updated tool call with properly typed arguments
        """
        if "parameters" not in tool_spec or "properties" not in tool_spec["parameters"]:
            return tool_call
            
        properties = tool_spec["parameters"]["properties"]
        
        # Standard type converters
        type_mapping = {
            "int": int,
            "str": str,
            "bool": bool,
            "float": float,
            "integer": int,
            "string": str,
            "boolean": bool,
            "number": float
        }
        
        for arg_name, arg_value in tool_call["arguments"].items():
            if arg_name in properties and "type" in properties[arg_name]:
                expected_type = properties[arg_name]["type"]
                
                if expected_type in type_mapping:
                    converter = type_mapping[expected_type]
                    try:
                        # Only convert if types don't match
                        if not isinstance(arg_value, converter):
                            print(f"{Fore.BLUE}Converting argument '{arg_name}' from {type(arg_value).__name__} to {expected_type}{Style.RESET_ALL}")
                            tool_call["arguments"][arg_name] = converter(arg_value)
                    except (ValueError, TypeError) as e:
                        print(f"{Fore.RED}Type conversion error for '{arg_name}': {str(e)}{Style.RESET_ALL}")
                        # Keep original value if conversion fails
                        pass
                        
        return tool_call
    
    def run(self, user_input: str) -> str:
        print(f"{Fore.WHITE}{Style.BRIGHT}=== Starting agent run with user input: '{user_input}' ==={Style.RESET_ALL}")
        # Add user input to conversation history
        self.conversation_history.append({"role": "user", "content": user_input})

        # Build the messages list for OpenAI API
        messages = [
            {"role": "system", "content": self.system_prompt + "\n\n" + self.format_tools_for_prompt()}
        ]
        
        # Add conversation history
        for message in self.conversation_history:
            messages.append({"role": message["role"], "content": message["content"]})

        # Get response from language model
        print(f"{Fore.CYAN}Calling language model...{Style.RESET_ALL}")
        response = self.model(model="gpt-4o", messages=messages)
        model_response = response.choices[0].message.content
        print(f"{Fore.CYAN}Received response from language model ({len(model_response)} chars){Style.RESET_ALL}")

        # Extract tool calls from response
        tool_calls = self.extract_tool_calls(model_response)
        
        # If no tool calls, return the response directly
        if not tool_calls:
            print(f"{Fore.GREEN}No tool calls needed. Returning response.{Style.RESET_ALL}")
            final_response = model_response
        else:
            # Execute tools and collect results
            tool_results = []
            for i, tool_call in enumerate(tool_calls):
                print(f"{Fore.MAGENTA}{Style.BRIGHT}Executing tool call {i+1}/{len(tool_calls)}{Style.RESET_ALL}")
                result = self.execute_tool(tool_call)
                tool_results.append({
                    "tool": tool_call.get("name"),
                    "arguments": tool_call.get("arguments"),
                    "result": result
                })
            
            # Format tool results
            results_text = "Tool results:\n"
            for res in tool_results:
                result_str = str(res["result"])
                if isinstance(res["result"], (dict, list)):
                    try:
                        result_str = json.dumps(res["result"], indent=2)
                    except:
                        pass
                    
                results_text += f"- {res['tool']}{json.dumps(res['arguments'])}: {result_str}\n"
            
            print(f"{Fore.BLUE}Formatted tool results{Style.RESET_ALL}")
            
            # Create a new message with original response and tool results
            messages.append({"role": "assistant", "content": model_response})
            messages.append({"role": "user", "content": results_text})
            
            # Get final response from language model with tool results
            print(f"{Fore.CYAN}Calling language model with tool results...{Style.RESET_ALL}")
            final_response_obj = self.model(model="gpt-4o", messages=messages)
            final_response = final_response_obj.choices[0].message.content
            print(f"{Fore.CYAN}Received final response from language model ({len(final_response)} chars){Style.RESET_ALL}")
        
        # Add final response to conversation history
        self.conversation_history.append({
            "role": "assistant", 
            "content": final_response
        })
        
        print(f"{Fore.WHITE}{Style.BRIGHT}=== Agent run completed ===={Style.RESET_ALL}")
        return final_response
        
    def reset_conversation(self):
        """Clear the conversation history."""
        print(f"{Fore.GREEN}Conversation history reset{Style.RESET_ALL}")
        self.conversation_history = []

In [166]:
REACT_SYSTEM_PROMPT = """
# ReAct Agent: Reasoning → Acting Framework  (FIRST-TURN VERSION)

You are an AI assistant that solves problems in a strict loop:
**Thought → Action → (STOP and wait)**  
Only after the system supplies an <observation> may you think again.

--------------------------------------------------------------------
## How You Operate - first turn

1. **THOUGHT** - explain your reasoning and decide which tool to call.  
2. **ACTION** - emit exactly one <tool_call> block in valid JSON.  
3. **STOP** - output nothing after </tool_call>.  
   *Do NOT produce <observation> or <response> until the host returns an observation.*

--------------------------------------------------------------------
## Available Tools

You can invoke any of the functions listed below:

<tools>
__TOOLS__
</tools>

--------------------------------------------------------------------
## Tool-Calling Format  (follow precisely)

<tool_call>{"name": "function_name", "arguments": {"param1": value1, "param2": value2}}</tool_call>

*Example - if one tool is*  
{"name":"compute_sum","parameters":{"properties":{"a":{"type":"int"},"b":{"type":"int"}}}}

→ valid call:  
<tool_call>{"name":"compute_sum","arguments":{"a":10,"b":20}}</tool_call>

--------------------------------------------------------------------
## First-Turn Template you MUST follow

<thought>…your reasoning here…</thought>  
<tool_call>{"name":"<one-of-the-tool-names>","arguments":{…}}</tool_call>

*(Nothing after </tool_call>.)*

--------------------------------------------------------------------
## What happens next

- The system executes the call and sends you:  
  <observation>{…tool result…}</observation>
- You will then start the next cycle (Thought → …).  
- When fully certain, finish with:  
  <thought>I now know the final answer.</thought>  
  <response>…answer…</response>

--------------------------------------------------------------------
## Important Guidelines

- Always begin with a <thought> tag.  
- One tool per action; multiple tools → multiple cycles.  
- JSON must match the schema exactly (types, key names).  
- If no tool is required, state why and answer directly:  

  <thought>This question can be answered directly…</thought>  
  <response>…answer…</response>

- **Never** output <response> in the first turn if you used a tool.  
- **Never** add text after your <tool_call>; simply stop.

--------------------------------------------------------------------
## Context
    you dont have memory, so this is your scratchpad. based on it to take your next decision.

    __SCRATCHPAD__

"""

In [167]:
tools = [
    {
        "name": "calculate_area",
        "description": "Calculate the area of a rectangle",
        "parameters": {
            "properties": {
                "length": {"type": "float"},
                "width": {"type": "float"}
            }
        }
    },
    {
        "name": "get_current_weather",
        "description": "Get the current weather for a location",
        "parameters": {
            "properties": {
                "location": {"type": "string"},
                "unit": {"type": "string"}
            }
        }
    }
]

# Convert tools to JSON string
tools_json = json.dumps(tools, indent=2)

In [168]:
@tool
def add_two_numbers(a: int, b: int) -> int:
    """Used to add two numbers together"""
    return a + b

@tool
def calculate_area_of_rectangle(length: float, width: float) -> float:
    
    """Used to calculate the area of a rectangle"""
    return length * width

In [169]:
add_two_numbers.specification

'{"name": "add_two_numbers", "description": "Used to add two numbers together", "parameters": {"properties": {"a": {"type": "int"}, "b": {"type": "int"}}}}'

In [170]:
tools = [add_two_numbers, calculate_area_of_rectangle]
tools_mapping = {tool.name: tool for tool in tools}

tools_specifications = "".join([tool.specification for tool in tools])


print(tools_specifications)

{"name": "add_two_numbers", "description": "Used to add two numbers together", "parameters": {"properties": {"a": {"type": "int"}, "b": {"type": "int"}}}}{"name": "calculate_area_of_rectangle", "description": "Used to calculate the area of a rectangle", "parameters": {"properties": {"length": {"type": "float"}, "width": {"type": "float"}}}}


In [171]:
import os
import google.generativeai as genai

In [172]:
genai.configure(api_key=os.environ["GOOGLE_API_KEY"])
        
llm = genai.GenerativeModel(
  model_name="gemini-1.5-flash",
  generation_config={"temperature": 0}
)

In [173]:
def transform_response_to_dict(response_content: str) -> dict:
    """
    Extracts all <tool_call> JSON blocks from the response_content and returns them as a list of dicts.
    Handles errors gracefully.
    """
    import re
    import json

    tool_calls = []
    try:
        # Find all <tool_call>...</tool_call> blocks
        matches = re.findall(r"<tool_call>\s*(\{.*?\})\s*</tool_call>", response_content, re.DOTALL)
        if not matches:
            raise ValueError("No <tool_call> JSON found in response.")

        for match in matches:
            try:
                tool_calls.append(json.loads(match))
            except json.JSONDecodeError as e:
                tool_calls.append({
                    "name": "JSONDecodeError",
                    "message": f"Error decoding JSON: {str(e)}",
                    "raw": match
                })
        return tool_calls
    except Exception as e:
        return [{
            "name": type(e).__name__,
            "message": str(e),
            "stack": None
        }]

In [174]:
def tool_result(transformed_response):
  for tool_call in transformed_response:
    tool_name = tool_call.get("name")
    tool = tools_mapping.get(tool_name)
    if tool:
      return tool.execute(**tool_call.get("arguments", {}))
    else:
      raise(f"Tool {tool_name} not found in tools_mapping")

In [176]:
def run(prompt):
  scratchpad = f"<question>{prompt}<\question>"
  while True:
    PROMPT = REACT_SYSTEM_PROMPT.replace("__TOOLS__", tools_specifications).replace("__SCRATCHPAD__", scratchpad)
    output = llm.generate_content(PROMPT).text
    scratchpad = scratchpad + "\n" + output
    if "<response>" not in output:
      transformed_response = transform_response_to_dict(output)
      r = tool_result(transformed_response)
      scratchpad = scratchpad + f"<observation>{r}</observation>"
    else:
      print(scratchpad)
      return
    print(scratchpad+'\n')

run("The sum of 10 and 20 is the width of a rectangle that is 100 units long. What is the area of the rectangle?")



<question>The sum of 10 and 20 is the width of a rectangle that is 100 units long. What is the area of the rectangle?<\question>
<thought>The problem requires two steps. First, I need to add 10 and 20 using the `add_two_numbers` tool.  Then, I'll use the result as the width in the `calculate_area_of_rectangle` tool to compute the area.</thought>
<tool_call>{"name": "add_two_numbers", "arguments": {"a": 10, "b": 20}}</tool_call>
<observation>30</observation>

<question>The sum of 10 and 20 is the width of a rectangle that is 100 units long. What is the area of the rectangle?<\question>
<thought>The problem requires two steps. First, I need to add 10 and 20 using the `add_two_numbers` tool.  Then, I'll use the result as the width in the `calculate_area_of_rectangle` tool to compute the area.</thought>
<tool_call>{"name": "add_two_numbers", "arguments": {"a": 10, "b": 20}}</tool_call>
<observation>30</observation>
<thought>The `add_two_numbers` tool returned 30.  Now I will use this as th